In [1]:
import os
import csv
import pandas as pd
import numpy as np
import statistics
import json
import dill
import wget
from scipy.fftpack import fft
from Bio import Entrez
from tqdm import tqdm
from Bio import SeqIO
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, label_binarize
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier
from sklearn import metrics
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

### Setup

Primeiramente é feita a filtragem dos Dados iniciais.

In [ ]:
os.makedirs('./data/raw', exist_ok=True)

classes = ['LTR', 'LINE', 'SINE', 'TIR', 'MITE', 'Helitron']

file_paths = os.listdir('./data/raw')
print(file_paths)

for name in classes:
    if f'TEAnnotationFinal_{name}.gff3' not in file_paths:
        wget.download(f'http://apte.cp.utfpr.edu.br/te-annotation/zea_mays/TEAnnotationFinal_{name}.gff3',
                      out=f'./data/raw/')

file_paths = os.listdir('./data/raw')

print(file_paths)

In [ ]:
data_df = pd.DataFrame()

if not os.path.isfile('./data/classes.csv'):
    data = []

    for path in file_paths:
        with open(f'./data/raw/{path}', 'r') as f:
            reader = csv.reader(f, delimiter='\t')

            class_name = path.split('.')[0].split('_')[1]

            for row in reader:
                if (row[6] == '+'):
                    data.append([class_name] + row)

    header = ['Class', 'Chr', 'Source Annotation', 'Class/Order/Superfamily', 'Start', 'End', 'Score', 'Strand', 'Phase', 'Attributes']

    data_df = pd.DataFrame(data, columns=header)
    data_df = data_df[['Class', 'Chr', 'Start', 'End']]

    data_df.to_csv('./data/classes.csv')

else:
    data_df = pd.read_csv('./data/classes.csv')

In [ ]:
data_df['Start'] = data_df['Start'].astype('int64')
data_df['End'] = data_df['End'].astype('int64')

Em sequência será feita a extração dos genomas por meio de Biopython

In [ ]:
id_dict = {"LR618874.1": "Chr_1.txt", "LR618875.1": "Chr_2.txt", "LR618876.1": "Chr_3.txt", "LR618877.1": "Chr_4.txt", 
           "LR618878.1": "Chr_5.txt", "LR618879.1": "Chr_6.txt", "LR618880.1": "Chr_7.txt", "LR618881.1": "Chr_8.txt", 
           "LR618882.1": "Chr_9.txt", "LR618883.1": "Chr_10.txt", "AY506529.1":"Chr_Mt.txt", "X86563.2": "Chr_Pt.txt"}

Entrez.email = "pedro.guilherme2305@usp.br"

os.makedirs('./data/sequences', exist_ok=True)

for id in tqdm(id_dict, total=len(id_dict)):
    if not os.path.isfile(f'./data/sequences/{id_dict[id]}'):
        stream = Entrez.efetch(db="nuccore", id=id, rettype="fasta")

        with open(f"./data/sequences/{id_dict[id]}", "w") as file:
            file.write(stream.read())
        stream.close()

### Processamento

In [ ]:
def generate_sequences(data_df: pd.DataFrame):
    chromosomes_to_keep = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'Mt', 'Pt']
    data_df = data_df.query("Chr in @chromosomes_to_keep") 

    aux_list = []

    for chromosome in chromosomes_to_keep:
        rows = data_df.query(f"Chr == '{chromosome}'").to_dict(orient="records")
        record = SeqIO.read(f"./data/sequences/Chr_{chromosome}.txt", "fasta")
        for row in tqdm(rows, total=len(rows)):
            aux_dict = dict()

            aux_dict['Chr'] = row['Chr']
            aux_dict['Sequence'] = record[row['Start']:row['End']].seq
            aux_dict['Class'] = row['Class']

            if aux_dict['Sequence'] == '': aux_dict['Sequence'] = np.nan

            aux_list.append(aux_dict)
    
    return aux_list

### Feauture Extraction - Accumulated Nucle Frequency Fourier

In [ ]:
def feature_extraction(spectrum, spectrumTwo):
    features = []

    average = sum(spectrum)/len(spectrum)
    features.append(average)
    ###################################
    median = np.median(spectrum)
    features.append(median)
	###################################
    maximum = np.max(spectrum)
    features.append(maximum)
    ###################################
    minimum = np.min(spectrum)
    features.append(minimum)
    ###################################
    peak = (len(spectrum)/3)/(average)
    features.append(peak)
    ###################################
    peak_two = (len(spectrumTwo)/3)/(np.mean(spectrumTwo))
    features.append(peak_two)
    ###################################
    standard_deviation = np.std(spectrum) # standard deviation
    features.append(standard_deviation)
    ###################################
    standard_deviation_pop = statistics.stdev(spectrum) # population sample standard deviation 
    features.append(standard_deviation_pop)
    ###################################
    percentile15 = np.percentile(spectrum, 15)
    features.append(percentile15)
    ###################################
    percentile25 = np.percentile(spectrum, 25)
    features.append(percentile25)
    ###################################
    percentile50 = np.percentile(spectrum, 50)
    features.append(percentile50)
    ###################################
    percentile75 = np.percentile(spectrum, 75)
    features.append(percentile75)
    ###################################
    amplitude = maximum - minimum
    features.append(amplitude)
    ###################################
    # mode = statistics.mode(spectrum)
    ###################################
    variance = statistics.variance(spectrum)
    features.append(variance)
    ###################################
    interquartile_range = np.percentile(spectrum, 75) - np.percentile(spectrum, 25)
    features.append(interquartile_range)
    ###################################
    semi_interquartile_range = (np.percentile(spectrum, 75) - np.percentile(spectrum, 25))/2 
    features.append(semi_interquartile_range)
    ###################################
    coefficient_of_variation = standard_deviation/average
    features.append(coefficient_of_variation)
    ###################################
    skewness = (3 * (average - median))/standard_deviation
    features.append(skewness)   
    ###################################
    kurtosis = (np.percentile(spectrum, 75) - np.percentile(spectrum, 25)) / (2 * (np.percentile(spectrum, 90) - np.percentile(spectrum, 10))) 
    features.append(kurtosis)
    ###################################
    return features


def accumulated_nucle_frequency_fourier(seq):
    
    seq = seq.upper()
    features = []
    spectrum = []
    spectrumTwo = []
    mapping = []
    A = 0
    C = 0
    T = 0
    G = 0
    for i in range(len(seq)):
        if seq[i] == 'A':
            A += 1
            mapping.append(A / (i + 1))
        elif seq[i] == 'C':
            C += 1
            mapping.append(C / (i + 1))
        elif seq[i] == 'T' or seq[i] == 'U':
            T += 1
            mapping.append(T / (i + 1))
        else:
            G += 1
            mapping.append(G / (i + 1))
    Fmap = fft(mapping)
    for i in range(len(mapping)):
        specTotal = (abs(Fmap[i])**2)
        specTwo = (abs(Fmap[i]))
        spectrum.append(specTotal)
        spectrumTwo.append(specTwo)
    
    features = feature_extraction(spectrum, spectrumTwo)

    return features

In [ ]:
def generate_features(final_df: pd.DataFrame, columns: list):
    features_dict = {}

    if not os.path.isfile('./data/features.json'):
        sequence_list = final_df['Sequence'].to_list()

        features_list = []
        for seq in tqdm(sequence_list, total=len(sequence_list)):
            features_list.append(accumulated_nucle_frequency_fourier(seq))

        features_list = np.array(features_list)
 
        features_dict = {}
        for i in tqdm(range(len(columns))):
            features_dict[columns[i]] = list(features_list[:, i])

        with open('./data/features.json', 'w') as f: 
            json.dump(features_dict, f)

    else:
        with open('./data/features.json', 'r') as f: 
            features_dict = json.load(f)
    
    return features_dict

In [2]:
columns = ['average', 'median', 'maximum', 'minimum', 'peak', 'none_levated_peak', 'sample_standard_deviation', 'population_standard_deviation', \
        'percentile15', 'percentile25', 'percentile50', 'percentile75', 'amplitude', 'variance', 'interquartile_range', 'semi_interquartile_range', \
        'coefficient_of_variation', 'skewness', 'kurtosis']

if not os.path.isfile('./data/final.csv'):
    aux_list = generate_sequences(data_df)

    final_df = pd.DataFrame(aux_list)
    final_df = final_df.dropna()
    final_df.head()

    features_dict = generate_features(final_df, columns)

    for column in columns:
        final_df[column] = features_dict[column]

    final_df = final_df.dropna()
    final_df.head()

    final_df.to_csv('./data/final.csv')

else:
    final_df = pd.read_csv('./data/final.csv')

/tmp/ipykernel_1173/3046178905.py:23: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  final_df = pd.read_csv('./data/final.csv')


### Machine Learning

In [3]:
final_df = final_df[[column for column in final_df.columns if column != 'Unnamed: 0']]

In [4]:
final_df.groupby('Class')['Chr'].count()

Class
Helitron     24513
LINE         11058
LTR         189795
MITE         26094
SINE          3697
TIR          83113
Name: Chr, dtype: int64

In [5]:
class_list = final_df['Class'].to_list()
le = LabelEncoder()
y = le.fit_transform(class_list)

columns = ['average', 'median', 'maximum', 'minimum', 'peak', 'none_levated_peak', 'sample_standard_deviation', 'population_standard_deviation', \
            'percentile15', 'percentile25', 'percentile50', 'percentile75', 'amplitude', 'variance', 'interquartile_range', 'semi_interquartile_range', \
            'coefficient_of_variation', 'skewness', 'kurtosis']

x = final_df[columns].values

In [6]:
x_resample, y_resample = NearMiss(sampling_strategy={0:15000, 2:15000, 3:15000, 5:15000}).fit_resample(x, y)
x_resample, y_resample = SMOTE(sampling_strategy={1: 15000, 4:15000}).fit_resample(x_resample, y_resample)
y_resample = label_binarize(y_resample, classes=[0, 1, 2, 3, 4, 5])
x_train, x_test, y_train, y_test = train_test_split(x_resample, y_resample, test_size=0.3)

In [ ]:
classifier = make_pipeline(StandardScaler(), OneVsRestClassifier(RandomForestClassifier(criterion='gini', n_estimators=100)))

In [ ]:
classifier_list = []
criterion_list = ['gini', 'log_loss']
tree_num_list = [50, 100, 200]
for tree_num in tree_num_list:
    for criterion in criterion_list:
        classifier = make_pipeline(StandardScaler(), OneVsRestClassifier(RandomForestClassifier(criterion=criterion, n_estimators=tree_num)))
        classifier_list.append({'criterion':criterion, 'tree_num': tree_num, 'classifier': classifier})

In [8]:
path = './data/classifiers/'
os.makedirs(path, exist_ok=True)

for classifier in tqdm(classifier_list, total=len(classifier_list)):
    
    file_name = f'classifier_{classifier['criterion']}_{classifier['tree_num']}.pkl'

    if file_name in list(os.listdir('./data/classifiers/')):
        with open(f'{path}/{file_name}', 'rb') as f: classifier['classifier'] = dill.load(f)
    else:
        classifier['classifier'].fit(x_train, y_train)
        with open(f'{path}/{file_name}', 'wb') as f: dill.dump(classifier['classifier'], f)

100%|██████████| 4/4 [00:03<00:00,  1.01it/s]


In [9]:
for classifier in classifier_list:
    y_pred = classifier['classifier'].predict(x_test)
    print(classification_report(y_test, y_pred, target_names=['Helitron', 'LINE', 'LTR', 'MITE', 'SINE', 'TIR']))

/home/paoebom/micromamba/envs/bioinfo/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

    Helitron       0.93      0.80      0.86      4489
        LINE       0.91      0.72      0.81      4504
         LTR       0.98      0.73      0.83      4430
        MITE       0.90      0.76      0.83      4521
        SINE       0.93      0.56      0.70      4520
         TIR       0.94      0.75      0.83      4536

   micro avg       0.93      0.72      0.81     27000
   macro avg       0.93      0.72      0.81     27000
weighted avg       0.93      0.72      0.81     27000
 samples avg       0.72      0.72      0.72     27000



/home/paoebom/micromamba/envs/bioinfo/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

    Helitron       0.93      0.80      0.86      4489
        LINE       0.92      0.72      0.81      4504
         LTR       0.98      0.73      0.83      4430
        MITE       0.91      0.76      0.83      4521
        SINE       0.93      0.57      0.70      4520
         TIR       0.94      0.74      0.83      4536

   micro avg       0.93      0.72      0.81     27000
   macro avg       0.94      0.72      0.81     27000
weighted avg       0.94      0.72      0.81     27000
 samples avg       0.72      0.72      0.72     27000



/home/paoebom/micromamba/envs/bioinfo/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

    Helitron       0.93      0.79      0.85      4489
        LINE       0.92      0.72      0.81      4504
         LTR       0.97      0.73      0.83      4430
        MITE       0.91      0.75      0.82      4521
        SINE       0.95      0.58      0.72      4520
         TIR       0.93      0.73      0.82      4536

   micro avg       0.93      0.72      0.81     27000
   macro avg       0.93      0.72      0.81     27000
weighted avg       0.93      0.72      0.81     27000
 samples avg       0.72      0.72      0.72     27000

              precision    recall  f1-score   support

    Helitron       0.93      0.79      0.85      4489
        LINE       0.92      0.72      0.81      4504
         LTR       0.98      0.73      0.83      4430
        MITE       0.91      0.75      0.82      4521
        SINE       0.95      0.59      0.73      4520
         TIR       0.93      0.73      0.82      4536

   micro avg       0.93

/home/paoebom/micromamba/envs/bioinfo/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [26]:
y_pred = classifier.predict(x_test)

In [27]:
print(classification_report(y_test, y_pred, target_names=['Helitron', 'LINE', 'LTR', 'MITE', 'SINE', 'TIR']))

              precision    recall  f1-score   support

    Helitron       0.69      0.30      0.41      4450
        LINE       0.69      0.40      0.51      4566
         LTR       0.61      0.11      0.18      4449
        MITE       0.57      0.23      0.33      4561
        SINE       0.81      0.48      0.60      4466
         TIR       0.55      0.18      0.27      4508

   micro avg       0.68      0.28      0.40     27000
   macro avg       0.65      0.28      0.39     27000
weighted avg       0.65      0.28      0.39     27000
 samples avg       0.28      0.28      0.28     27000



/home/paoebom/micromamba/envs/bioinfo/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:", metrics.precision_score(y_test, y_pred, average='weighted'))
print("Recall:", metrics.recall_score(y_test, y_pred, average='weighted'))

Accuracy: 0.27585185185185185
Precision: 0.6532132781253848
Recall: 0.28125925925925926


In [ ]:
with open('./data/classifier.pkl', 'wb') as f: dill.dump(classifier, f) 